# 실습 08: 표 검출 및 데이터 추출

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/leecks1119/document_ai_lecture/blob/master/notebooks/Lab08_표검출.ipynb)

## 🎯 학습 목표
- 이미지에서 표 영역 검출
- 표 구조 분석 (행/열)
- 표 데이터를 DataFrame으로 변환

## ⏱️ 소요 시간: 40분
## 📊 난이도: ⭐⭐⭐⭐☆


In [ ]:
# 한글 폰트 설치 및 설정 (Colab용)
!apt-get install -y fonts-nanum fonts-nanum-coding fonts-nanum-extra
!fc-cache -fv

# matplotlib 한글 폰트 설정
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=10)
plt.rc('font', family=fontprop.get_name())
plt.rc('axes', unicode_minus=False)

print("✅ 한글 폰트 설정 완료!")


In [ ]:
# 환경 설정
!pip install -q opencv-python-headless paddlepaddle paddleocr
!apt-get install -y tesseract-ocr tesseract-ocr-kor

import cv2
import numpy as np
import pandas as pd
from paddleocr import PaddleOCR
import matplotlib.pyplot as plt


In [ ]:
# 표가 포함된 샘플 이미지 생성
from PIL import Image, ImageDraw, ImageFont

# 한글 폰트 로드
try:
    font_title = ImageFont.truetype("C:\\Windows\\Fonts\\malgunbd.ttf", 28)  # 굵은 글씨
    font_cell = ImageFont.truetype("C:\\Windows\\Fonts\\malgun.ttf", 18)
except:
    try:
        font_title = ImageFont.truetype("/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf", 28)
        font_cell = ImageFont.truetype("/usr/share/fonts/truetype/nanum/NanumGothic.ttf", 18)
    except:
        font_title = ImageFont.load_default()
        font_cell = ImageFont.load_default()

img = Image.new('RGB', (800, 400), color='white')
draw = ImageDraw.Draw(img)

# 제목
draw.text((30, 20), "견적서", fill='black', font=font_title)

# 표 그리기
table_data = [
    ["품목", "수량", "단가", "금액"],
    ["노트북", "10", "1,500,000", "15,000,000"],
    ["모니터", "20", "300,000", "6,000,000"],
    ["키보드", "30", "50,000", "1,500,000"],
]

x_start, y_start = 50, 70
cell_width = 180
cell_height = 40

# 셀 그리기
for i, row in enumerate(table_data):
    for j, cell in enumerate(row):
        x = x_start + j * cell_width
        y = y_start + i * cell_height
        # 테두리
        draw.rectangle([x, y, x+cell_width, y+cell_height], outline='black', width=2)
        # 텍스트
        draw.text((x+10, y+10), cell, fill='black', font=font_cell)

img.save('table_doc.jpg')
print("✅ 표 이미지 생성 완료!")
img


In [ ]:
# OpenCV로 표 영역 검출
img = cv2.imread('table_doc.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 이진화
_, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)

# 수평선 검출
horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1))
horizontal_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)

# 수직선 검출
vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 40))
vertical_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, vertical_kernel, iterations=2)

# 표 구조 결합
table_structure = cv2.add(horizontal_lines, vertical_lines)

# 시각화
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0].set_title('원본 이미지')
axes[0].axis('off')

axes[1].imshow(horizontal_lines, cmap='gray')
axes[1].set_title('수평선 검출')
axes[1].axis('off')

axes[2].imshow(vertical_lines, cmap='gray')
axes[2].set_title('수직선 검출')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("✅ 표 구조 검출 완료!")


In [ ]:
# PaddleOCR로 텍스트 추출 (최신 API)
ocr = PaddleOCR(use_textline_orientation=True, lang='korean')
result = ocr.predict('table_doc.jpg')

# 좌표 기준으로 정렬 (위→아래, 왼쪽→오른쪽)
ocr_results = []
for line in result[0]:
    bbox = line[0]
    text = line[1]
    conf = line[2]
    
    # 중심점 계산
    x_center = (bbox[0][0] + bbox[2][0]) / 2
    y_center = (bbox[0][1] + bbox[2][1]) / 2
    
    ocr_results.append({
        'text': text,
        'x': x_center,
        'y': y_center,
        'confidence': conf
    })

# Y 좌표로 먼저 정렬 (행), 그 다음 X 좌표로 정렬 (열)
ocr_results_sorted = sorted(ocr_results, key=lambda r: (r['y'], r['x']))

print("\n📋 추출된 텍스트 (순서대로):")
for i, item in enumerate(ocr_results_sorted, 1):
    print(f"{i:2d}. {item['text']:20s} (신뢰도: {item['confidence']:.2%})")


In [ ]:
# DataFrame으로 변환 (4열 구조)
texts = [item['text'] for item in ocr_results_sorted]

# 4개씩 묶어서 DataFrame 생성
rows = []
for i in range(0, len(texts), 4):
    if i + 4 <= len(texts):
        rows.append(texts[i:i+4])

df = pd.DataFrame(rows[1:], columns=rows[0])

print("\n📊 표 데이터 (DataFrame):")
print("="*70)
print(df.to_string(index=False))
print("="*70)

# CSV로 저장
df.to_csv('extracted_table.csv', index=False, encoding='utf-8-sig')
print("\n✅ CSV 저장 완료: extracted_table.csv")
